In [17]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)

### Create a Pydantic model for Evaluation

In [29]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [2]:
with open('summary.txt', 'r', encoding='utf-8') as file:
    summary = file.read()

In [3]:
name = 'Kaushik'

In [4]:
from pypdf import PdfReader


pdfreader = PdfReader('linkedin.pdf')
linkedin = ''
for page in pdfreader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [13]:
system_prompt = f'''
You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions about {name}'s career, background, experience and skills. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Linkedin profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
You must greet every user as {name} and with a very smaill introduction as {name} at the beginning of the conversation. \
If you do not know an answer please politely say so
'''
system_prompt += f'\n\n## Summary: {summary}\n\n## LinkedIn Profile: {linkedin}\n\n'
system_prompt += f'With this context, please chat with the user, always stay in character as {name}'

In [5]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
print(evaluator_system_prompt)

In [7]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [21]:
import os

from openai import OpenAI

gemini = OpenAI(
    api_key=os.getenv('GOOGLE_API_KEY'),
    base_url='https://generativelanguage.googleapis.com/v1beta/openai/'
)

In [22]:
openai = OpenAI()

In [35]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{'role':'system', 'content':evaluator_system_prompt}] + [{'role':'user', 'content': evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model='gemini-2.0-flash', messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed


In [14]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [32]:
def chat(message, history):
    messages = [{'role':'system', 'content':system_prompt}] + history + [{'role':'user', 'content':message}]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    reply = response.choices[0].message.content
    evaluation = evaluate(reply, message, history)
    if evaluation.is_acceptable:
        print('Passed evaluation - returning reply')
    else:
        print('Failed evaluation - retrying')
        print(evaluation.feedback)
        rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
import gradio as gr

gr.ChatInterface(chat, type='messages').launch()